In [28]:
TRAIN_PATH = 'data/original/train.jsonl'
DEV_PATH = 'data/original/dev.jsonl'
TEST_PATH = 'data/original/test.jsonl'

In [29]:
import pandas as pd
import json

def read_jsonl_to_dataframe(file_path):
    data = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                json_obj = json.loads(line)
                data.append(json_obj)
            except json.JSONDecodeError as e:
                print(f"Skipping invalid JSON: {e}")

    df = pd.DataFrame(data)

    return df

In [30]:
df_train = read_jsonl_to_dataframe(TRAIN_PATH)
df_dev = read_jsonl_to_dataframe(DEV_PATH)
df_test = read_jsonl_to_dataframe(TEST_PATH)

# Processing data

In [32]:
from underthesea import word_tokenize
import regex as re

def tokenize(text):
    return word_tokenize(text, format="text")

def preprocess(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", " ", text) # remove punctuation
    text = re.sub(r"\s+", " ", text) # remove extra space
    text = text.strip()
    return text

def post_process(text):

    # text = tokenize(text)
    text = preprocess(text)

    return text

In [33]:
def get_all_span_aspect(text, label):
    
    for span in label:
        start, end, aspect_sentiment = span
        aspect = text[start:end]
        
        yield aspect, aspect_sentiment

df_train['all_span_aspect'] = df_train.apply(lambda row: list(get_all_span_aspect(row['text'], row['labels'])), axis=1)
df_test['all_span_aspect'] = df_test.apply(lambda row: list(get_all_span_aspect(row['text'], row['labels'])), axis=1)
df_dev['all_span_aspect'] = df_dev.apply(lambda row: list(get_all_span_aspect(row['text'], row['labels'])), axis=1)

In [34]:
# apply post process to all_span_aspect
df_train['all_span_aspect'] = df_train.apply(lambda row: [(post_process(aspect), sentiment) for aspect, sentiment in row['all_span_aspect']], axis=1)
df_test['all_span_aspect'] = df_test.apply(lambda row: [(post_process(aspect), sentiment) for aspect, sentiment in row['all_span_aspect']], axis=1)
df_dev['all_span_aspect'] = df_dev.apply(lambda row: [(post_process(aspect), sentiment) for aspect, sentiment in row['all_span_aspect']], axis=1)

In [35]:
def process_data_frame(df):
    # new_df = pd.DataFrame()
    df['text'] = df['text'].apply(lambda x: post_process(x))
    # df['labels'] = df['labels']
    # df['all_span_aspect'] = df['all_span_aspect']
    return df

In [36]:
df_train = process_data_frame(df_train)
df_dev = process_data_frame(df_dev)
df_test = process_data_frame(df_test)

In [37]:
# save data
import os

def save_data_to_csv(df, folder, filename):
    if not os.path.exists(folder):
        os.mkdir(folder)

    with open(f'{folder}/{filename}', 'w', encoding='utf-8') as file:
        df.to_json(file, force_ascii=False)

save_data_to_csv(df_train, 'data/process_data', 'train.json')
save_data_to_csv(df_dev, 'data/process_data', 'dev.json')
save_data_to_csv(df_test, 'data/process_data', 'test.json')

# Combine data

In [1]:
import pandas as pd
import json

def read_json_to_dataframe(filename):

    with open(filename, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    df = pd.DataFrame(data)

    return df

df_train = read_json_to_dataframe('data/process_data/train.json')
df_dev = read_json_to_dataframe('data/process_data/dev.json')
df_test = read_json_to_dataframe('data/process_data/test.json')

In [3]:
def find_aspect(text, all_span_aspect):
    
    for span_aspect, sentiment in all_span_aspect:
        
        # find start and end of aspect in text
        start = text.find(span_aspect)
        end = start + len(span_aspect)

        # print(f"start: {start}, end: {end}, aspect: {span_aspect}, sentiment: {sentiment}")

        yield start, end, sentiment

df_train['labels'] = df_train.apply(lambda row: list(find_aspect(row['text'], row['all_span_aspect'])), axis=1)
df_dev['labels'] = df_dev.apply(lambda row: list(find_aspect(row['text'], row['all_span_aspect'])), axis=1)
df_test['labels'] = df_test.apply(lambda row: list(find_aspect(row['text'], row['all_span_aspect'])), axis=1)

In [4]:
del df_train['all_span_aspect']
del df_dev['all_span_aspect']
del df_test['all_span_aspect']

In [59]:
df_train.text = df_train.text.apply(lambda x: tokenize(x))
df_dev.text = df_dev.text.apply(lambda x: tokenize(x))
df_test.text = df_test.text.apply(lambda x: tokenize(x))

In [61]:
save_data_to_csv(df_train, 'data/process_data_final', 'train.json')
save_data_to_csv(df_dev, 'data/process_data_final', 'dev.json')
save_data_to_csv(df_test, 'data/process_data_final', 'test.json')

In [63]:
def print_data(index):
    print(df_train.iloc[index].text)
    # print(df_train.iloc[index].labels)
    # print(df_train.iloc[index].all_span_aspect)
    print(df_train.iloc[index].labels)
    print("\n=============================================================================================\n")

# loop through all data
for i in range(len(df_train)):
    print_data(i)

pin sài tầm 50 h cho pin 100 100 camera ổn tất_cả đều ok nhân_viên thế_giới di_động trần_văn_thời cà_mau nhiệt_tình và vui_vẻ chúc các ae sức khỏe tốt và phục_ok hoài_nha
[(0, 31, 'BATTERY#POSITIVE'), (32, 41, 'CAMERA#POSITIVE'), (42, 55, 'GENERAL#POSITIVE'), (56, 169, 'SER&ACC#POSITIVE')]


lag và hao_pin là cái tóm_tắt về máy_sam làm tệ quá không bằng mấy con tàu cùng phân_khúc
[(0, 3, 'PERFORMANCE#NEGATIVE'), (8, 15, 'BATTERY#NEGATIVE'), (44, 104, 'GENERAL#NEGATIVE')]


tất_cả đều ổn ngoại_trừ lúc máy nóng lên thì pin tụt nhanh hơn tụt quần_haizz cam chụp cũng gọi_là tầm_trung nếu dùng lướt_web bình_thường thì có_thể dùng được cả ngày
[(78, 108, 'CAMERA#NEUTRAL'), (109, 167, 'BATTERY#POSITIVE'), (28, 71, 'BATTERY#NEGATIVE')]


ok mua máy ở tgdd chính_sách đổi trả rất tốt rất yên_tâm khi mua ở tgdd quản_lý ở 15 a đường bà hôm phường 13 quận 6 nhẹ_nhàng và vui tính_nhiên_viên ai cũng nhiệt_tình hài_lòng về cách chăm_sóc khách_hàng của các bạn
[(18, 216, 'SER&ACC#POSITIVE')]


kiểu_dán

# End